In [27]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, json
import random
import numpy as np
import tqdm
from utils_batch import InfillingModel
from torch.nn.functional import log_softmax
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc



device = "cuda:1" # for GPU usage or "cpu" for CPU usage

In [28]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce GTX 1050 Ti with Max-Q Design
Memory Usage:
Allocated: 0.7 GB
Cached:    2.2 GB


In [29]:
def plot_roc_curve(human_scores, gpt_scores):
    # Data
    A, B = human_scores, gpt_scores
    # Combine scores and true labels
    scores = A + B
    labels = [0] * len(A) + [1] * len(B)
    # check amount of NaNs in scores
    if np.isnan(human_scores).any():
        print("Warning: NaNs in scores")
        print(np.argwhere(np.isnan(scores)))
    if np.isnan(gpt_scores).any():
        print("Warning: NaNs in scores")
        print(np.argwhere(np.isnan(scores)))
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(labels, scores)
    # Calculate AUC (Area Under Curve)
    roc_auc = auc(fpr, tpr)
    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve: Open-gen w/ GPT3.5-Reddit w prompts' )
    plt.legend(loc="lower right")
    plt.show()
    # what is the TPR for FPR = 0.1?
    for idx, fpr_ in enumerate(fpr):
        if fpr_ > 0.1:
            print(f"TPR at 1% FPR: {tpr[idx]:.4f}")
            break
    return roc_auc, tpr[idx]

In [30]:
model_name = "NinedayWang/PolyCoder-160M"
# NinedayWang/PolyCoder-0.4B
# NinedayWang/PolyCoder-2.7B
# model_name = 'codeparrot/codeparrot'
PyCodeGPT = AutoModelForCausalLM.from_pretrained( model_name ).to(device)
PyCodeGPT_tokenizer = AutoTokenizer.from_pretrained( model_name )

In [31]:
# from transformers import LlamaForCausalLM, LlamaTokenizer
# model_name = "/data/xianjun/project/llama/7B_hf/"
# model = LlamaForCausalLM.from_pretrained( model_name ).half().to(device) #.half() to use FP16
# model.eval() 
# PyCodeGPT = model
# PyCodeGPT_tokenizer = LlamaTokenizer.from_pretrained( model_name ) #.half() to use FP16

In [32]:
from transformers import AutoConfig
# Load the model's configuration
config = AutoConfig.from_pretrained(model_name)
# Get the default max_length
max_length = config.max_position_embeddings
max_length

2048

In [33]:
# give an input, return the logits of input tokens
inputs = 'this is a test'
truncate_ratio=0.9
def get_logprob1(inputs):
    input_ids = PyCodeGPT_tokenizer.encode(inputs, return_tensors='pt').to(device)
    input_ids = input_ids[:, :max_length]
    with torch.no_grad():
        output = PyCodeGPT(input_ids)
    logits = output[0]
    # Assuming the `logits` tensor contains the output from the model
    log_probs = log_softmax(logits, dim=-1)
    # Select the log probabilities for the specific tokens in the input
    input_log_probs = log_probs[0, torch.arange(log_probs.size(1)), input_ids[0]]
    # Multiply by -1 to get the negative log probabilities
    neg_log_probs = -input_log_probs
    neg_log_probs = neg_log_probs.cpu().numpy().tolist()
    return np.average( neg_log_probs[int( truncate_ratio*len(neg_log_probs)): ] ), neg_log_probs

In [34]:
#### merge the datasets
with open('FIM/validation/codebert_val_FIM_human_combined_line_8_per_56.jsonl', 'r') as f:
    data1  = [json.loads(line) for line in f]
len(data1)

353

In [35]:
# sample couple of lines and json dump them
machine_text = data1[2]
print(json.dumps(machine_text, indent=4))

{
    "code": "n, k = map(int, input().split())\n\na = list(map(int, input().split()))\n\na.sort()\n\ntotal_a = sum(a)\n\nif total_a < n:\n    print(-1)\nelse:\n    ans = 0\n    current_a = 0\n    for i in range(len(a)):\n        current_a += a[i]\n        ans += 1\n\n        if current_a >= n:\n            break\n\n    print(ans)",
    "label": 1,
    "label_name": "machine_generated",
    "difficulty": "competition",
    "original_source": "results\\gemma-7b-it-apps_competition_207.jsonl",
    "problem_id": "3862",
    "question": "Provide me the Python3 codes for solving the question: Sasha and Kolya decided to get drunk with Coke, again. This time they have k types of Coke. i-th type is characterised by its carbon dioxide concentration $\\frac{a_{i}}{1000}$. Today, on the party in honour of Sergiy of Vancouver they decided to prepare a glass of Coke with carbon dioxide concentration $\\frac{n}{1000}$. The drink should also be tasty, so the glass can contain only integer number of l

In [36]:
human_sample = data1[6]
human_text = human_sample['code']
print(json.dumps(human_sample, indent=4))

{
    "code": "import sys\n\ndef gcd(l):\n\tif len(l)==0:\n\t\treturn 0\n\tif len(l)==1:\n\t\treturn l[0]\n\tif len(l)==2:                                   \n\t\tif l[1]==0:\n\t\t\treturn l[0]  \n\t\treturn gcd([l[1],l[0]%l[1]])\n\treturn gcd([gcd(l[:-1]),l[-1]])\n\t\t\ndef brute_force(l1,l2,l,sol):                   \n\tif len(l)==0:\n\t\tg1=gcd(l1)\n\t\tg2=gcd(l2)\n\t\treturn g1==1 and g2==1,sol\n\t\n\tres,s=brute_force(l1+[l[0]],l2,l[1:],sol+[1])\n\tif res:\n\t\treturn True,s\n\treturn brute_force(l1,l2+[l[0]],l[1:],sol+[2])\n\ndef factor(n):\n\tres=[]\n\ti=2\n\twhile i*i<=n:\n\t\tif n%i==0:\n\t\t\tres.append(i)\n\t\twhile n%i==0:\n\t\t\tn=int(n/i)\n\t\ti+=1\n\n\tif n!=1:\n\t\tres.append(n)\n\n\treturn res\n\ndef dumpsol(sol):\n\tfor v in sol:\n\t\tprint(v,end=' ')\n\nn=int(sys.stdin.readline())\nl=sys.stdin.readline().strip().split(\" \")[0:n]\nl=[int(x) for x in l]\nif n<12:\n\tret,sol=brute_force([],[],l,[])\n\tif ret:\n\t\tprint(\"YES\")\n\t\tdumpsol(sol)\n\telse:\n\t\tprint(\"

In [37]:
len(data1[0]['FIM_code'])

56

In [38]:
avg_neg_log_probs, neg_log_probs = get_logprob1(human_text)
neg_log_probs[-500:]

[16.161951065063477,
 3.5731394290924072,
 0.681315541267395,
 13.048877716064453,
 10.80954360961914,
 20.374858856201172,
 14.935730934143066,
 15.403397560119629,
 13.82616901397705,
 19.6192626953125,
 16.476253509521484,
 15.183921813964844,
 1.4620287418365479,
 1.5568031072616577,
 12.343795776367188,
 10.939208030700684,
 21.5939884185791,
 20.667089462280273,
 11.97369384765625,
 15.549126625061035,
 24.074020385742188,
 1.4298579692840576,
 0.7834327816963196,
 19.570222854614258,
 15.123907089233398,
 7.0347185134887695,
 0.0013233008794486523,
 13.265477180480957,
 14.065176963806152,
 13.979874610900879,
 15.221598625183105,
 22.916425704956055,
 13.157648086547852,
 14.758708953857422,
 10.99094009399414,
 16.433635711669922,
 16.281391143798828,
 14.480605125427246,
 2.407822847366333,
 3.7958102226257324,
 7.933293342590332,
 11.59392261505127,
 20.09247398376465,
 3.8899354934692383,
 17.644363403320312,
 14.079073905944824,
 7.288999557495117,
 12.274282455444336,
 17

In [39]:
# give an input, return the logits of input tokens
truncate_ratio=0.99
def get_logprob(inputs):
    input_ids = PyCodeGPT_tokenizer.encode(inputs, return_tensors='pt').to(device)
    input_ids = input_ids[:, :max_length]
    with torch.no_grad():
        output = PyCodeGPT(input_ids)
    logits = output[0]
    # Assuming the `logits` tensor contains the output from the model
    log_probs = log_softmax(logits, dim=-1)
    # Select the log probabilities for the specific tokens in the input
    input_log_probs = log_probs[0, torch.arange(log_probs.size(1)), input_ids[0]]
    # Multiply by -1 to get the negative log probabilities
    neg_log_probs = -input_log_probs
    neg_log_probs = neg_log_probs.cpu().numpy().tolist()
    return np.average( neg_log_probs[int( truncate_ratio*len(neg_log_probs)): ] )

In [40]:
prob_all = []
for id, ins in tqdm.tqdm(enumerate(data1), total=len(data1)):
    temp = []
    if len( ins['FIM_code']) > 1:
        original_score = get_logprob(ins['code'])
        miu_scores = []
        for i in range( len(ins['FIM_code'] ) ):
            # if not  ins['FIM_code'][i]['completed']:
            #     continue
            one_regen = ins['FIM_code'][i]['text']
            miu_scores.append(get_logprob( one_regen ))
        miu_scores_average_score = np.ma.average( miu_scores )
        dx = original_score - miu_scores_average_score
        prob_all.append(dx)

  1%|▏         | 5/353 [01:19<1:32:19, 15.92s/it]


KeyboardInterrupt: 

In [ ]:

    # gold_prob_all = []
    # for id, ins in tqdm.tqdm(enumerate(data1), total=len(data1)):
    #     temp = []
    #     if len( ins['human_gen_text']['choices'] ) > 1:
    #         original_score = get_logprob( ins['gold_completion'] ) - get_logprob( ins['human_prefix_prompt'] )
    #         miu_scores = []
    #         for i in range( len(ins['human_gen_text']['choices'] ) ):
    #             one_regen = ins['human_prefix_prompt'] + ins['human_gen_text']['choices'][i]['message']['content']
    #             miu_scores.append( get_logprob( one_regen ) - get_logprob( ins['human_prefix_prompt'] ) )
    #         miu_scores_average_score = np.average( miu_scores )
    #         dx = original_score - miu_scores_average_score
    #         gold_prob_all.append( dx )
   

In [ ]:
prob_all

In [ ]:
print(len(prob_all))

In [ ]:
human_scores = []
machine_scores = []
for ind, ins in tqdm.tqdm(enumerate(data1), total=len(data1)):
    #check for nan
    if np.isnan(prob_all[ind]):
        continue
    if ins['label'] == 0:
        human_scores.append(prob_all[ind])
    else:
        machine_scores.append(prob_all[ind])
        

In [ ]:
print(len(human_scores))
print(human_scores)

In [ ]:
print(len(machine_scores))
print(machine_scores)

In [ ]:
# plot and give different colors
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(sorted(human_scores), label='human')
plt.plot(sorted(machine_scores), label='gpt')
plt.legend()
plt.show()

In [ ]:


plot_roc_curve(human_scores, machine_scores)

In [ ]:
# count negative values
print( len( [i for i in human_scores if i < 0] ) )
print( len( [i for i in machine_scores if i < 0] ) )

In [ ]:
print(gold_prob_all)